In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# Project modules
from utils import *
from utils_visualisation import *
from Data_Handler import ClimbingDataset
from models import *
from engine import *

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchmetrics.classification import MulticlassAccuracy


# Image filters
import kornia.filters as KF

#Bayesian Hyperparam
import optuna

/home/fillies/Documents/moon/kilter/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.__version__
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [ ]:
def objective(trial: optuna.Trial):
    blur_kernel_size = trial.suggest_categorical("blur_kernel_size", [1, 3, 5, 7]) 
    blur_sigma = trial.suggest_float("blur_sigma", 0.5, 3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    hidden_units_CNN = trial.suggest_categorical("hidden_units_CNN", [8, 16, 32, 64])
    hidden_units_classifier = trial.suggest_categorical("hidden_units_classifier", [16, 32, 64, 128, 256])
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)   # log scale
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    # Load Data 
    # necessary bc blur is hyperparameter dependent
    
    transform = KF.GaussianBlur2d(kernel_size=(blur_kernel_size, blur_kernel_size), sigma=(blur_sigma, blur_sigma), border_type="constant")
    two_class_dataset = ClimbingDataset(board_names=["12 x 12 with kickboard Square"],
                                map=True, 
                                transform=transform,
                                label_filter=[5, 14],  # 6a+ & 7c
                                max_samples=10000 
    )
    num_classes = two_class_dataset.num_classes
    
    
    train_size = int(0.8 * len(two_class_dataset))  # 80% train
    test_size = len(two_class_dataset) - train_size

    # Deterministic split (set generator seed for reproducibility)
    train_dataset, test_dataset = random_split(
        two_class_dataset,
        [train_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    # Wrap in DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    
    #Model
    model_0 = shallowCNN(input_shape= 4,
                     hidden_units_CNN= hidden_units_CNN, 
                     hidden_units_classifier=  hidden_units_classifier, 
                     output_shape=num_classes 
                     ).to(device)


    # Loss and Optimizer
    loss_fn = torch.nn.BCEWithLogitsLoss()
    acc_fn = Multiclass_accuracy()# MulticlassAccuracy(num_classes=num_classes).to(device)
    optimizer = optim.Adam(model_0.parameters(), lr=lr, weight_decay=weight_decay )
    
    
    model, history = train_model(
        model=model_0,
        train_loader=train_loader,
        test_loader=test_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        acc_fn=acc_fn,
        device=device,
        epochs=5,
        patience=7
    )
    
    return history["test_acc"][-1]
    
    
    
    



In [6]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("Best trial:")
print("  Value:", study.best_trial.value)
print("  Params:", study.best_trial.params)

[I 2025-10-01 09:01:41,714] A new study created in memory with name: no-name-1f7d8f77-288d-4910-b310-d1b28d5ac875


/home/fillies/Documents/moon/kilter/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Epoch 0000 | Train loss: 0.68950 | Train acc: 0.00000 | Test loss: 0.68547 | Test acc: 0.00000
Timing: total 15.802 s | train 14.568 s | eval 1.234 s

Epoch 0001 | Train loss: 0.68466 | Train acc: 0.00000 | Test loss: 0.68214 | Test acc: 0.00000
Timing: total 14.811 s | train 13.801 s | eval 1.010 s

Epoch 0002 | Train loss: 0.67915 | Train acc: 0.00000 | Test loss: 0.67607 | Test acc: 0.00000
Timing: total 10.000 s | train 9.013 s | eval 0.987 s

Epoch 0003 | Train loss: 0.67251 | Train acc: 0.00000 | Test loss: 0.66856 | Test acc: 0.00000
Timing: total 8.566 s | train 7.464 s | eval 1.101 s


[I 2025-10-01 09:02:59,488] Trial 0 finished with value: 0.0 and parameters: {'blur_kernel_size': 1, 'blur_sigma': 0.864983129635226, 'batch_size': 64, 'hidden_units_CNN': 32, 'hidden_units_classifier': 128, 'lr': 1.9739415868185522e-05, 'weight_decay': 1.946758711498223e-05}. Best is trial 0 with value: 0.0.



Epoch 0004 | Train loss: 0.66419 | Train acc: 0.00000 | Test loss: 0.65908 | Test acc: 0.00000
Timing: total 8.986 s | train 8.227 s | eval 0.759 s


...


Epoch 0000 | Train loss: 0.68346 | Train acc: 0.03077 | Test loss: 0.67152 | Test acc: 0.02679
Timing: total 3.637 s | train 2.973 s | eval 0.665 s

Epoch 0001 | Train loss: 0.65675 | Train acc: 0.01172 | Test loss: 0.63963 | Test acc: 0.00781
Timing: total 2.574 s | train 1.980 s | eval 0.593 s

Epoch 0002 | Train loss: 0.61838 | Train acc: 0.01758 | Test loss: 0.59407 | Test acc: 0.05588
Timing: total 2.454 s | train 1.804 s | eval 0.650 s

Epoch 0003 | Train loss: 0.56650 | Train acc: 0.41576 | Test loss: 0.53591 | Test acc: 0.57661
Timing: total 3.801 s | train 2.995 s | eval 0.807 s

Epoch 0004 | Train loss: 0.50523 | Train acc: 0.57404 | Test loss: 0.47108 | Test acc: 0.57661
Timing: total 4.229 s | train 3.335 s | eval 0.894 s


[I 2025-10-01 09:06:55,146] Trial 3 finished with value: 0.576612902539117 and parameters: {'blur_kernel_size': 5, 'blur_sigma': 0.9965253961703988, 'batch_size': 32, 'hidden_units_CNN': 8, 'hidden_units_classifier': 256, 'lr': 1.4300833425940423e-05, 'weight_decay': 1.7890017575792018e-06}. Best is trial 1 with value: 0.6916402642216001.
/home/fillies/Documents/moon/kilter/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Epoch 0000 | Train loss: 0.09529 | Train acc: 0.62515 | Test loss: 0.07725 | Test acc: 0.57878
Timing: total 27.269 s | train 25.751 s | eval 1.518 s

Epoch 0001 | Train loss: 0.04943 | Train acc: 0.74892 | Test loss: 0.07710 | Test acc: 0.65473
Timing: total 23.764 s | train 22.487 s | eval 1.277 s

Epoch 0002 | Train loss: 0.04216 | Train acc: 0.79691 | Test loss: 0.04643 | Test acc: 0.78550
Timing: total 29.531 s | train 26.068 s | eval 3.462 s

Epoch 0003 | Train loss: 0.04142 | Train acc: 0.80699 | Test loss: 0.09574 | Test acc: 0.62459
Timing: total 20.290 s | train 18.664 s | eval 1.626 s

Epoch 0004 | Train loss: 0.04033 | Train acc: 0.81401 | Test loss: 0.09047 | Test acc: 0.64803
Timing: total 27.134 s | train 25.946 s | eval 1.188 s


[I 2025-10-01 09:09:33,650] Trial 4 finished with value: 0.64803004690579 and parameters: {'blur_kernel_size': 1, 'blur_sigma': 1.3093955476433876, 'batch_size': 64, 'hidden_units_CNN': 64, 'hidden_units_classifier': 256, 'lr': 0.009422492478932074, 'weight_decay': 1.113443504747652e-05}. Best is trial 1 with value: 0.6916402642216001.


Best trial:
  Value: 0.6916402642216001
  Params: {'blur_kernel_size': 7, 'blur_sigma': 2.016913904493669, 'batch_size': 32, 'hidden_units_CNN': 16, 'hidden_units_classifier': 32, 'lr': 0.00731542357675252, 'weight_decay': 0.0008924768970092811}
